In [2]:
import numpy as np
from math import sin, cos
import math
from numpy import log
from typing import List, Tuple
from numpy.testing import assert_almost_equal

# forward mode

In [3]:
class ValueFwd:
    """ stores a single scalar value and its gradient """
    def __init__(self, data, diff = 0):
        self.v = data
        self.d = diff

    def __add__(l, r):
        """ support for + on ValueFwd datatype.

         l + r returns f, such that f.v = l.v + r.v and f.d = f' = df/dx.

        Args:
            l (ValueFwd): ValueFwd to the left of +.
            r (ValueFwd): ValueFwd to the right of +.

        Returns:
            ValueFwd with updated value and gradient.
        """
        return ValueFwd(l.v + r.v, l.d + r.d)

    def __sub__(l, r):
        """ support for - on ValueFwd datatype.

         l - r returns f, such that f.v = l.v - r.v and f.d = f' = df/dx.

        Args:
            l (ValueFwd): ValueFwd to the left of -.
            r (ValueFwd): ValueFwd to the right of -.

        Returns:
            ValueFwd with updated value and gradient.
        """
        return ValueFwd(l.v - r.v, l.d - r.d)

    def __mul__(l, r):
        """ support for * on ValueFwd datatype.

         l * r returns f, such that f.v = l.v * r.v and f.d = f' = df/dx.

        Args:
            l (ValueFwd): ValueFwd to the left of *.
            r (ValueFwd): ValueFwd to the right of *.

        Returns:
            ValueFwd with updated value and gradient.
        """
        return ValueFwd(l.v * r.v, l.d * r.v + l.v * r.d)

    def __truediv__(l, r):
        """ support for / on ValueFwd datatype.

         l / r returns f, such that f.v = l.v / r.v and f.d = f' = df/dx.

        Args:
            l (ValueFwd): ValueFwd to the left of /.
            r (ValueFwd): ValueFwd to the right of /.

        Returns:
            ValueFwd with updated value and gradient.
        """
        return ValueFwd(l.v / r.v, ((l.d * r.v - l.v * r.d) / r.v**2))

    def sin(self):
        """ support for self.sin() on ValueFwd datatype.

        Args:
            self (ValueFwd): ValueFwd.

        Returns:
            returns ValueFwd f, such that f.v = sin(self.v) and f.d = f' = df/dx
        """
        return ValueFwd(math.sin(self.v), math.cos(self.v) * self.d)

    def cos(self):
        """ support for self.cos() on ValueFwd datatype.

        Args:
            self (ValueFwd): ValueFwd.

        Returns:
            returns ValueFwd f, such that f.v = cos(self.v) and f.d = f' = df/dx
        """
        return ValueFwd(math.cos(self.v), -math.sin(self.v) * self.d)

    def exp(self, b):
        """ support for b^self on ValueFwd datatype.

            self.exp(math.e) = e^self

        Args:
            self (ValueFwd): ValueFwd.
            b (Number): A numerical value.

        Returns:
            returns ValueFwd f, such that f.v = b^(self.v) and f.d = f' = df/dx
        """
        return ValueFwd(b**self.v, (b**self.v) * math.log(b) * self.d)

    def pow(self, p):
        """ support for pow on ValueFwd datatype.

            self.pow(3) = self^3

        Args:
            self (ValueFwd): ValueFwd.
            p (Number): A numerical value.

        Returns:
            returns ValueFwd f, such that f.v = (self.v)^p and f.d = f' = df/dx
        """
        return ValueFwd(self.v**p, p * (self.v**(p-1)) * self.d)

    def __repr__(self):
        return f"v: {self.v}, d:{self.d}"

# Test forward-mode automatic differentiation

Test function 1 is: `test_f1(x)`
$$f(x) = \sin\left( \frac{\sqrt{e^x + 2}}{2}\right)$$

The gradient for Test 1 is: `test_df1`
$$\frac{e^x \cos\left( \frac{\sqrt{e^x + 2}}{2} \right)}{4 \sqrt{e^x + 2}}$$


In [4]:
def test_f1(x):
    return ((x.exp(math.e) + ValueFwd(2)).pow(0.5)/ValueFwd(2)).sin()

def test_df1(x):
    ex = math.exp(x)
    num = ex*cos(math.sqrt(ex+2)/2)
    den = 4 * math.sqrt(ex+2)
    return num/den

## test Froward-mode Automatic Differentiation

In [5]:
assert_almost_equal(test_f1(ValueFwd(1, 1)).d, test_df1(1))

In [6]:
# Note: Above Assert worked, hence no Errors were returned! We'll check the values anyways

print(test_f1(ValueFwd(1, 1)).d)
print(test_df1(1))
print(test_f1(ValueFwd(2, 1)).d - test_df1(2))

# we see they're as close to each other in the order of a -16

0.14577682456620103
0.14577682456620103
1.3530843112619095e-16


# reverse mode

### Reverse-mode

We do two passes in the reverse mode of Automatic Differentiation. One forward and one backward.

### Implementation

Reverse mode is a little more complicated than the forward mode. Here gradients are accumulated during the backward pass. so, we also have to store the computation DAG(Directed Acyclic Graph) so that the backward pass can happen in the correct order.

In [7]:
class ValueBwd:
    """ stores a single scalar value and its gradient, updates gradient through back-prop """
    def __init__(self, data, prev = []):
        """ Value datatype that supports backpropagation on a single scalar variable.

        Args:
            data (Number): value of the variable/constant.
            prev (List[Tuple[ValueBwd, Number]]): List of node(ValueBwd), local gradient tuples that the current node depends on.
                Empty list if the computational graph contains a single node.
        Returns:
            ValueBwd.
        """
        self.v = data
        self.d = 0
        self._prev = prev

    def __add__(l, r):
        """ support for + on ValueBwd datatype.

        Args:
            l (ValueBwd): ValueBwd to the left of +.
            r (ValueBwd): ValueBwd to the right of +.

        Returns:
            ValueBwd with updated value and gradient.
        """
        return ValueBwd(l.v + r.v, [(l, 1), (r, 1)])

    def __sub__(l, r):
        """ support for - on ValueBwd datatype.

        Args:
            l (ValueBwd): ValueBwd to the left of -.
            r (ValueBwd): ValueBwd to the right of -.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        return ValueBwd(l.v - r.v, [(l, 1), (r, -1)])

    def __mul__(l, r):
        """ support for * on ValueBwd datatype.

        Args:
            l (ValueBwd): ValueBwd to the left of -.
            r (ValueBwd): ValueBwd to the right of -.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        return ValueBwd(l.v * r.v, [(l, r.v), (r, l.v)])

    def __truediv__(l, r):
        """ support for / on ValueBwd datatype.

        Args:
            l (ValueBwd): ValueBwd to the left of -.
            r (ValueBwd): ValueBwd to the right of -.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        return ValueBwd(l.v / r.v, [(l, 1/r.v), (r, -l.v/r.v**2)])

    def sin(self):
        """ support for sin(self) on ValueBwd datatype.

        Args:

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        return ValueBwd(math.sin(self.v), [(self, math.cos(self.v))])

    def cos(self):
        """ support for cos(self) on ValueBwd datatype.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        return ValueBwd(math.cos(self.v), [(self, -math.sin(self.v))])

    def exp(self, b):
        """ support for b^self on ValueBwd datatype.

        Args:
            b (Number):  A numerical value.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        return ValueBwd(b**self.v, [(self, b**self.v*math.log(b))])

    def pow(self, p):
        """ support for self^p on ValueBwd datatype.

        Args:
            p (ValueBwd):  A numerical value.

        Returns:
            ValueBwd: Current node of the computational graph.
        """
        return ValueBwd(self.v**p, [(self, p * (self.v**(p-1)))])

    def backward(self, d=1.0):
      """
      Updates local gradient and recursively calls backward on self._prev.

      Args:
          d (Number): The gradient value flowing back from the child node.
                      Defaults to 1.0 for the final output node.
      """
      self.d += d

      for parent_node, local_gradient in self._prev:
          gradient_to_pass_back = d * local_gradient
          parent_node.backward(gradient_to_pass_back)

    def __repr__(self):
        return f"v: {self.v}, d:{self.d}"

## Test reverse mode AD

In [8]:
def test_f1(x):
    return ((x.exp(math.e) + ValueBwd(2)).pow(0.5)/ValueBwd(2)).sin()

def test_df1(x):
    ex = math.exp(x)
    num = ex*cos(math.sqrt(ex+2)/2)
    den = 4 * math.sqrt(ex+2)
    return num/den

In [9]:
x = 2
inp = ValueBwd(x)
o = test_f1(inp)
o.backward(1)

In [10]:
assert_almost_equal(inp.d, test_df1(2))

In [11]:
print(test_df1(2) - inp.d)

# Assert returns no issues
# Same as above, they indeed are ",close enough", hence our back prop works perfectly

-1.3530843112619095e-16
